In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(

    # 256x256
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 128x128

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 64x64

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 32x32

    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 16x16

    # 🔥 Global Average Pooling
    nn.AdaptiveAvgPool2d((1, 1)),  # 256x1x1

    nn.Flatten(),                  # 256

    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),

    nn.Linear(128, 2)
)

# Test
x = torch.randn(1, 3, 256, 256)
print(model(x).shape)


torch.Size([1, 2])


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

train_dataset = datasets.ImageFolder("data/train", transform=train_transform)
val_dataset = datasets.ImageFolder("data/val", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
print("Number of training samples: ", len(train_dataset))

Number of training samples:  71071


In [ ]:
print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)


{'fake': 0, 'real': 1}
{'fake': 0, 'real': 1}


In [ ]:
print("CUDA available: ", torch.cuda.is_available())
print("GPU name: ", torch.cuda.get_device_name(0))

CUDA available:  True
GPU name:  NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
batch_size = 32


In [ ]:
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

num_epochs = 20
best_f1 = 0.0  # Initialisation cruciale

for epoch in range(num_epochs):
    # ================= TRAIN =================
    model.train()
    correct, total, running_loss = 0, 0, 0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        train_bar.set_postfix({"Loss": f"{loss.item():.4f}", "Acc": f"{100*correct/total:.2f}%"})

    train_acc = 100 * correct / total

    # ================= VALIDATION =================
    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Métriques
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary', pos_label=0
    )
    val_acc = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    val_loss /= len(val_loader)

    print(f"\n📊 Epoch {epoch+1} Summary:")
    print(f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}% | Val Loss: {val_loss:.4f}")
    print(f"🎯 Fake Class (0) -> Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}\n")

    # ================= CHECKPOINT =================
    if f1 > best_f1: 
        best_f1 = f1
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "f1_score": f1,
            "val_loss": val_loss
        }, "best_deepfake_detector.pt")
        print(f"🔥 Nouveau record F1-Score: {f1:.4f}! Modèle sauvegardé.\n")

Epoch 1/20: 100%|██████████| 2221/2221 [03:52<00:00,  9.57it/s, Loss=0.7267, Acc=70.18%]



📊 Epoch 1 Summary:
Train Acc: 70.18% | Val Acc: 69.07% | Val Loss: 0.5744

🔥 Nouveau meilleur modèle sauvegardé !


Epoch 2/20: 100%|██████████| 2221/2221 [03:52<00:00,  9.53it/s, Loss=0.4299, Acc=73.94%]



📊 Epoch 2 Summary:
Train Acc: 73.94% | Val Acc: 75.19% | Val Loss: 0.5028

🔥 Nouveau meilleur modèle sauvegardé !


Epoch 3/20: 100%|██████████| 2221/2221 [03:53<00:00,  9.53it/s, Loss=0.5615, Acc=76.35%]


...


📊 Epoch 18 Summary:
Train Acc: 89.67% | Val Acc: 89.37% | Val Loss: 0.2577



Epoch 19/20: 100%|██████████| 2221/2221 [03:53<00:00,  9.52it/s, Loss=0.2648, Acc=89.98%]



📊 Epoch 19 Summary:
Train Acc: 89.98% | Val Acc: 82.80% | Val Loss: 0.4082



Epoch 20/20: 100%|██████████| 2221/2221 [03:53<00:00,  9.51it/s, Loss=0.2162, Acc=90.32%]



📊 Epoch 20 Summary:
Train Acc: 90.32% | Val Acc: 77.28% | Val Loss: 0.5608

